In [1]:
import pandas as pd
file = pd.read_csv('D:\\NLP\Assignment\\L1\\movie_comments.csv',encoding='UTF-8')
comments = file['comment'].tolist()
print(comments[0:10])

['吴京意淫到了脑残的地步，看了恶心想吐', '首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮番上场，视物理逻辑于不顾，不得不说有钱真好，随意胡闹', '吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋律，为了煽情而煽情，让人觉得他是个大做作、大谎言家。（7.29更新）片子整体不如湄公河行动，1.整体不够流畅，编剧有毒，台词尴尬；2.刻意做作的主旋律煽情显得如此不合时宜而又多余。', '凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。', '中二得很', '“犯我中华者，虽远必诛”，吴京比这句话还要意淫一百倍。', '脑子是个好东西，希望编剧们都能有。', '三星半，实打实的7分。第一集在爱国主旋律内部做着各种置换与较劲，但第二集才真正显露吴京的野心，他终于抛弃李忠志了，新增外来班底让硬件实力有机会和国际接轨，开篇水下长镜头和诸如铁丝网拦截RPG弹头的细节设计都让国产动作片重新封顶，在理念上，它甚至做到《绣春刀2》最想做到的那部分。', '开篇长镜头惊险大气引人入胜 结合了水平不俗的快剪下实打实的真刀真枪 让人不禁热血沸腾 特别弹簧床架挡炸弹 空手接碎玻璃 弹匣割喉等帅得飞起！就算前半段铺垫节奏散漫主角光环开太大等也不怕 作为一个中国人 两个小时弥漫着中国强大得不可侵犯的氛围 还是让那颗民族自豪心砰砰砰跳个不停。', '15/100吴京的冷峰在这部里即像成龙，又像杰森斯坦森，但体制外的同类型电影，主角总是代表个人，无能的政府需要求助于这些英雄才能解决难题，体现的是个人的价值，所以主旋律照抄这种模式实际上是有问题的。我们以前嘲笑个人英雄主义，却没想到捆绑爱国主义的全能战士更加难以下咽。']


D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import re,string
from zhon import hanzi
def token(stri):
    punc = string.punctuation + hanzi.punctuation
    t = re.findall('\w+',str(stri))
    return ''.join(a for a in t if a not in punc)    
comments_clean = token(comments)
print(comments_clean[0:15])

吴京意淫到了脑残的地步看了恶心


In [3]:
import jieba 
def cut(string):
    return list(jieba.cut(string))
comments_words = cut(comments_clean)
print(comments_words[0:10])


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\canaan\AppData\Local\Temp\jieba.cache
Loading model cost 0.621 seconds.
Prefix dict has been built succesfully.


['吴京', '意淫', '到', '了', '脑残', '的', '地步', '看', '了', '恶心']


In [8]:
print(comments_words[1]+comments_words[2])
print(type(comments_words))

意淫到
<class 'list'>


In [9]:
from collections import Counter
words_count = Counter(comments_words)
words_count.most_common(10)
#frequencies = [f for f in words_count.most_common(10)]

[('的', 328256),
 ('了', 102409),
 ('是', 73421),
 ('我', 50510),
 ('都', 36251),
 ('很', 34755),
 ('看', 33853),
 ('电影', 33638),
 ('也', 32064),
 ('和', 31291)]

In [10]:
token = comments_words
def prob_1(word):
    return words_count[word]/len(token)
prob_1('好看')

0.0017941751635849222

In [26]:
def token_2(token):
    token_2 = []
    for i in range(len(token[:-1])):
        t = token[i]+token[i+1]
        token_2.append(t)
    return token_2
token_2 = token_2(token)

In [31]:
print(token_2[-5:])

['感觉就是', '就是他', '他男朋友', '男朋友真让人', '真让人恶心']


In [29]:
words_count_2 = Counter(token_2)
def prob_2(word1,word2):
    if word1+word2 in words_count_2:
        return words_count_2[word1+word2]/len(token_2)
    else:
        return (prob_1(word1)+prob_1(word2))/2
prob_2('晚上','好看')
    

0.0009254693849385005

In [36]:
def sentence_prob(sentence):
    words = cut(sentence)
    sentence_prob=1
    for i,w in enumerate(sentence[:-1]):
        next_2 = sentence[i+1]
        prob = prob_2(w,next_2)
        sentence_prob *=prob
    return sentence_prob
print(sentence_prob('电影又精彩又令人感到兴奋'))
print(sentence_prob('电影又精彩又令人感到无聊'))
print(sentence_prob('电影又精彩又好看'))
print(sentence_prob('电影又精彩又无聊'))


5.902905409002553e-46
1.5344059099420902e-46
2.4673269932002946e-30
2.061900138767372e-32


In [32]:
import random
def create_grammer(gram,split):
    grammer = {}
    for line in gram.split('\n'):
        if not line.strip(): continue
        exp,stmt = line.split(split)
        grammer[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammer

def generate(gram,target):
    if target in gram:
        expand_next = random.choice(gram[target])
        expanded_next = [generate(gram,t) for t in expand_next]
        return ''.join([e for e in expanded_next])        
    else:
        return target
    
def generate_n(gram,target,n):
    g_n = []
    for i in range(n):
        if target in gram:
            expand_next = random.choice(gram[target])
            expanded_next = [generate(gram,t) for t in expand_next]
            t = ''.join([e for e in expanded_next]) 
        else:
            t = target
        g_n.append(str(t))
    return g_n

In [33]:
human = """
human = 自己 寻找 活动
自己 =    我 | 俺 | 我们 
寻找 = 看看 | 找找 | 想找点
活动 = 乐子 | 玩的
"""
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""
grammer_human = create_grammer(human,'=')
grammer_host = create_grammer(host,'=')
generate_human = generate(grammer_human,'human')
generate_host = generate(grammer_host,'host')
generate_n_human = generate_n(grammer_human,'human',5)
print(generate_n_human)

['我找找乐子', '俺看看乐子', '我想找点玩的', '俺想找点玩的', '我们想找点乐子']


In [48]:
def generate_best(gram,target,n):
    sentences = generate_n(gram,target,n)
    sentences_prob = []
    for s in sentences:
        sentences_prob.append(sentence_prob(s))
    sentences_prob = list(enumerate(sentences_prob))
    sentences_prob.sort(key=lambda x:x[1],reverse=True)
    index = sentences_prob[0][0]
    return sentences[index]
print(generate_best(grammer_host,'host',5))

先生,你好我是1号,请问你要玩一玩赌博吗？


In [ ]:
Q: 这个模型有什么问题？ 你准备如何提升？
Ans: 尝试后发现由2-Gram model得出概率的准确性与句子长度呈负相关关系。可以用长的马尔可夫链模型尝试改进。